In [1]:
import pandas as pd
import numpy as np

In [2]:
wine = pd.read_csv("winemag-data-130k-v2.csv", index_col="id")

# Aperçu de la data

In [3]:
wine.head(3)

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
id,,,,,,,,,,,,,
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm


In [4]:
wine.count()

country                  129908
description              129971
designation               92506
points                   129971
price                    120975
province                 129908
region_1                 108724
region_2                  50511
taster_name              103727
taster_twitter_handle     98758
title                    129971
variety                  129970
winery                   129971
dtype: int64

On remarque que beaucoups de lignes dans différentes colonnes sont vides

# Nettoyage des données 

## Variety

Commençons par isoler les variétés de vin qui apparaissent à peu de reprises

In [5]:
wine['variety'].value_counts()

Pinot Noir                   13272
Chardonnay                   11753
Cabernet Sauvignon            9472
Red Blend                     8946
Bordeaux-style Red Blend      6915
                             ...  
Tempranillo-Tannat               1
Silvaner-Traminer                1
Pinot Gris-Gewürztraminer        1
Kinali Yapincak                  1
Loureiro-Arinto                  1
Name: variety, Length: 707, dtype: int64

On crée une nouvelle colonne avec les valeurs de chaque variété

In [6]:
wine['variety_count'] = wine.groupby('variety')['variety'].transform('count')
wine.head(1)

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,variety_count
id,,,,,,,,,,,,,,
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2360.0


Puis on remplace les variétés présente moins de 1000 fois par la valeur 'Other' à l'aide d'une colonne dédiée  

In [7]:
wine['other_variety'] = np.where(wine['variety_count']< 1000, 'Other', ['variety_ok'])
wine.head(1)

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,variety_count,other_variety
id,,,,,,,,,,,,,,,
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2360.0,variety_ok


In [8]:
wine.loc[wine['other_variety'] == 'Other', 'variety'] = 'Other'
wine['variety'].value_counts()

Other                         26648
Pinot Noir                    13272
Chardonnay                    11753
Cabernet Sauvignon             9472
Red Blend                      8946
Bordeaux-style Red Blend       6915
Riesling                       5189
Sauvignon Blanc                4967
Syrah                          4142
Rosé                           3564
Merlot                         3102
Nebbiolo                       2804
Zinfandel                      2714
Sangiovese                     2707
Malbec                         2652
Portuguese Red                 2466
White Blend                    2360
Sparkling Blend                2153
Tempranillo                    1810
Rhône-style Red Blend          1471
Pinot Gris                     1455
Champagne Blend                1396
Cabernet Franc                 1353
Grüner Veltliner               1345
Portuguese White               1159
Bordeaux-style White Blend     1066
Pinot Grigio                   1052
Gamay                       

Nos variétés sont standardisées, on peut supprimer les colonnes devenues inutiles

In [9]:
wine.drop(['variety_count', 'other_variety'], axis=1, inplace=True)

## Winery

Répétons l'opération avec la colonne vignoble cette fois-ci et un minimum de 10

In [10]:
wine['winery'].value_counts()

Wines & Winemakers    222
Testarossa            218
DFJ Vinhos            215
Williams Selyem       211
Louis Latour          199
                     ... 
Bernard Machado         1
Dominique Pabiot        1
Château Camplazens      1
Château Fontenelle      1
La Filice               1
Name: winery, Length: 16757, dtype: int64

In [11]:
wine['winery_count'] = wine.groupby('winery')['winery'].transform('count')
wine['other_winery'] = np.where(wine['winery_count']< 10, 'Other', ['winery'])
wine.loc[wine['other_winery'] == 'Other', 'winery'] = 'Other'

wine.drop(['winery_count', 'other_winery'], axis=1, inplace=True)
wine['winery'].value_counts()

Other                  40449
Wines & Winemakers       222
Testarossa               218
DFJ Vinhos               215
Williams Selyem          211
                       ...  
Sanctuary                 10
Brandl                    10
Espiritu de Chile         10
Collelceto                10
Quinta Vale do Armo       10
Name: winery, Length: 3705, dtype: int64

## Designation

A nouveau la même opération avec la désignation, en prenant un minimum de 5 

In [12]:
wine['designation'].value_counts()

Reserve                             2009
Estate                              1322
Reserva                             1259
Riserva                              698
Estate Grown                         621
                                    ... 
den Hoed Vineyard                      1
Cuvée Saint Rémi                       1
Calvario                               1
Rosé Pescador                          1
Lone Tree Vineyard Barrel Select       1
Name: designation, Length: 37976, dtype: int64

In [13]:
wine['designation_count'] = wine.groupby('designation')['designation'].transform('count')
wine['other_designation'] = np.where(wine['designation_count']< 5, 'Other', ['designation'])
wine.loc[wine['other_designation'] == 'Other', 'designation'] = 'Other'
wine.drop(['designation_count', 'other_designation'], axis=1, inplace=True)
wine['designation'].value_counts()

Other                       53588
Reserve                      2009
Estate                       1322
Reserva                      1259
Riserva                       698
                            ...  
Saulheimer Hölle Trocken        5
Moulin des Dames                5
Grand Réserve                   5
Cape Roca Fish                  5
Four Flags                      5
Name: designation, Length: 2968, dtype: int64

## Province

On continue avec la colonne Province et un seuil de 500

In [14]:
wine['province_count'] = wine.groupby('province')['province'].transform('count')
wine['other_province'] = np.where(wine['province_count']< 500, 'Other', ['province'])
wine.loc[wine['other_province'] == 'Other', 'province'] = 'Other'
wine.drop(['province_count', 'other_province'], axis=1, inplace=True)
wine['province'].value_counts()

California              36247
Other                   15864
Washington               8639
Bordeaux                 5941
Tuscany                  5897
Oregon                   5373
Burgundy                 3980
Northern Spain           3851
Piedmont                 3729
Mendoza Province         3264
Veneto                   2716
New York                 2688
Alsace                   2440
Northeastern Italy       2138
Loire Valley             1856
Sicily & Sardinia        1797
Champagne                1613
Southwest France         1503
South Australia          1349
Southern Italy           1349
Provence                 1346
Douro                    1281
Central Italy            1233
Catalonia                1164
Rhône Valley             1081
Beaujolais               1044
Mosel                    1039
Alentejano                937
Colchagua Valley          830
Marlborough               798
Virginia                  777
Tejo                      683
France Other              676
Burgenland

## Title

Analysons l'intitulé des vins

In [15]:
wine['title'].value_counts()

Gloria Ferrer NV Sonoma Brut Sparkling (Sonoma County)                     11
Korbel NV Brut Sparkling (California)                                       9
Segura Viudas NV Extra Dry Sparkling (Cava)                                 8
Segura Viudas NV Aria Estate Extra Dry Sparkling (Cava)                     7
Gloria Ferrer NV Blanc de Noirs Sparkling (Carneros)                        7
                                                                           ..
Cadence 2012 Tapteil Vineyard Red (Red Mountain)                            1
Mesa Del Sol 2011 Prima Rosso Red (Arroyo Seco)                             1
Beronia 2005 Reserva  (Rioja)                                               1
Bruno Paillard 2002 Assemblage Brut  (Champagne)                            1
Bouvet-Ladubay NV Signature Brut Chenin Blanc-Chardonnay (Vin Mousseux)     1
Name: title, Length: 118840, dtype: int64

On remarque deux choses, certains intitulés contiennent l'année de production du vin, et la plupart contiennent entre parenthèses des informations sur la région de  provenance qui sont redondantes avec les colonnes Province et Region_1 

## Year

Dans un premier temps nous allons extraire toutes les suites de 4 chiffres depuis la colonne Title pour les ajouter dans une nouvelle colonne Year

In [16]:
import re as re
def find_number(text):
    num = re.findall(r'[0-9]{4}',text)
    return " ".join(num)
wine['year_from_title']=wine['title'].apply(lambda x: find_number(x))
wine.head(1)

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,year_from_title
id,,,,,,,,,,,,,,
0,Italy,"Aromas include tropical fruit, broom, brimston...",Other,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2013


In [17]:
wine['year_from_title'].value_counts()

2013         15805
2012         15680
2014         15526
2011         12500
2010         12127
             ...  
2009 1000        1
2006 1905        1
1965             1
2011 1850        1
2005 1772        1
Name: year_from_title, Length: 386, dtype: int64

Certaines lignes contiennent plusieurs suites de 4 chiffres, pour en savoir plus on va créer une DataFrame à partir de cette colonne Year

In [18]:
import re
x = []
for i in wine['year_from_title']:
    x.append(re.findall("[0-2][0-9][0-9][0-9]", i))

In [19]:
pd.DataFrame(x).describe()

,0,1,2
count,125362,439,3
unique,89,111,2
top,2013,1865,1492
freq,15855,36,2


Dans la majorité des cas (125362 lignes), une seule année est présente, mais pour d'autres on en trouve 2 voire même 3 dans certains cas

In [20]:
pd.DataFrame(x)[0].unique()

array(['2013', '2011', '2012', '2010', '2007', '2009', '2008', '2014',
       '2015', None, '2016', '2004', '2003', '2006', '2001', '2005',
       '2002', '1887', '2000', '1999', '1991', '1997', '1996', '1877',
       '2017', '1995', '1872', '1637', '1868', '1898', '1492', '1998',
       '1852', '1994', '1992', '1840', '1929', '1912', '1875', '1976',
       '1964', '1848', '1870', '1856', '1983', '1967', '1990', '1988',
       '1827', '1860', '1850', '1000', '1980', '1987', '1989', '1993',
       '1969', '1882', '1935', '1503', '1821', '1973', '1978', '1965',
       '1968', '1947', '1963', '1070', '1985', '1927', '1904', '1847',
       '1982', '1986', '1752', '1789', '1607', '1621', '1919', '1957',
       '1966', '1984', '1961', '1845', '1952', '1150', '1941', '1974',
       '1934', '1945'], dtype=object)

In [21]:
pd.DataFrame(x)[1].unique()

array([None, '1105', '1423', '2013', '1939', '2002', '2009', '1865',
       '2640', '1601', '2010', '1846', '1769', '2131', '2007', '2012',
       '1234', '1904', '2011', '1919', '1905', '1573', '1000', '1979',
       '1300', '1762', '1973', '1907', '1194', '1147', '1827', '2008',
       '1772', '1023', '1910', '1836', '1903', '1810', '1868', '2401',
       '1918', '1924', '2003', '1040', '2015', '1866', '1894', '1698',
       '1908', '1619', '1610', '1028', '2005', '1815', '1735', '1927',
       '1149', '2900', '1988', '1861', '1869', '2014', '1998', '1955',
       '1912', '1895', '1100', '1840', '1614', '1879', '1906', '1888',
       '1953', '1844', '1970', '1940', '1753', '1913', '2221', '2004',
       '1620', '1479', '1935', '2016', '1843', '1870', '1522', '1685',
       '1875', '1951', '1947', '1850', '1628', '1871', '1550', '1265',
       '1880', '1070', '2000', '1909', '1867', '1914', '1860', '1975',
       '1930', '1995', '1886', '1922', '1780', '1885', '1916', '1637'],
      d

In [22]:
pd.DataFrame(x)[2].unique()

array([None, '1492', '1860'], dtype=object)

En analysant x on peut voir que (x)[2] n'est pas pertinent, en revanche (x)[0] et (x)[1] doivent être étudiés de manière plus approfondie pour déterminer quelle année est la bonne

Créons 2 nouvelles colonnes pour (x)[0] et (x)[1]

In [23]:
wine['year'] = pd.DataFrame(x)[0]
wine['year_2'] = pd.DataFrame(x)[1]
wine.head(1)

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,year_from_title,year,year_2
id,,,,,,,,,,,,,,,,
0,Italy,"Aromas include tropical fruit, broom, brimston...",Other,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2013,2013,None


In [24]:
wine['year'].unique()

array(['2013', '2011', '2012', '2010', '2007', '2009', '2008', '2014',
       '2015', None, '2016', '2004', '2003', '2006', '2001', '2005',
       '2002', '1887', '2000', '1999', '1991', '1997', '1996', '1877',
       '2017', '1995', '1872', '1637', '1868', '1898', '1492', '1998',
       '1852', '1994', '1992', '1840', '1929', '1912', '1875', '1976',
       '1964', '1848', '1870', '1856', '1983', '1967', '1990', '1988',
       '1827', '1860', '1850', '1000', '1980', '1987', '1989', '1993',
       '1969', '1882', '1935', '1503', '1821', '1973', '1978', '1965',
       '1968', '1947', '1963', '1070', '1985', '1927', '1904', '1847',
       '1982', '1986', '1752', '1789', '1607', '1621', '1919', '1957',
       '1966', '1984', '1961', '1845', '1952', '1150', '1941', '1974',
       '1934', '1945'], dtype=object)

In [25]:
wine['year_2'].unique()

array([None, '1105', '1423', '2013', '1939', '2002', '2009', '1865',
       '2640', '1601', '2010', '1846', '1769', '2131', '2007', '2012',
       '1234', '1904', '2011', '1919', '1905', '1573', '1000', '1979',
       '1300', '1762', '1973', '1907', '1194', '1147', '1827', '2008',
       '1772', '1023', '1910', '1836', '1903', '1810', '1868', '2401',
       '1918', '1924', '2003', '1040', '2015', '1866', '1894', '1698',
       '1908', '1619', '1610', '1028', '2005', '1815', '1735', '1927',
       '1149', '2900', '1988', '1861', '1869', '2014', '1998', '1955',
       '1912', '1895', '1100', '1840', '1614', '1879', '1906', '1888',
       '1953', '1844', '1970', '1940', '1753', '1913', '2221', '2004',
       '1620', '1479', '1935', '2016', '1843', '1870', '1522', '1685',
       '1875', '1951', '1947', '1850', '1628', '1871', '1550', '1265',
       '1880', '1070', '2000', '1909', '1867', '1914', '1860', '1975',
       '1930', '1995', '1886', '1922', '1780', '1885', '1916', '1637'],
      d

Considérons qu'aucun vin ne peut être plus vieux que 1900 et de fait remplaçons toutes les années antérieures à 1900 par la valeur de la colonne Year_2  

In [26]:
wine['year'] = np.where(wine['year'] < '1900', wine['year_2'], wine['year'])

In [27]:
wine['year'].unique()

array(['2013', '2011', '2012', '2010', '2007', '2009', '2008', '2014',
       '2015', None, '2016', '2004', '2003', '2006', '2001', '2005',
       '2002', '2000', '1999', '1991', '1997', '1996', '2017', '1995',
       '1998', '1994', '1992', '1929', '1912', '1976', '1964', '1983',
       '1967', '1990', '1988', '1980', '1987', '1989', '1993', '1969',
       '1935', '1973', '1978', '1965', '1968', '1947', '1963', '1985',
       '1927', '1904', '1982', '1986', '1919', '1957', '1966', '1984',
       '1961', '1952', '1941', '1974', '1934', '1945'], dtype=object)

Certaines années paraissent encore peu probables, pour vérifier, affichons les lignes ayant une colonne Year inférieure à l'année 2000 et une colonne Year_2 non nulle

In [28]:
wine.loc[(wine['year'] < '2000') & (wine['year_2'].notnull())]

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,year_from_title,year,year_2
id,,,,,,,,,,,,,,,,
7986,Italy,"This offers subdued aromas of vanilla, toasted...",Other,85,25.0,Tuscany,Vernaccia di San Gimignano,NaN,Kerin O’Keefe,@kerinokeefe,Guidi 1929 2012 Aurea Riserva (Vernaccia di S...,Other,Other,1929 2012,1929,2012
9728,Portugal,"With aromas of superripe fruit, this wood aged...",Other,90,NaN,Douro,NaN,NaN,Roger Voss,@vossroger,1912 Winemakers 2011 Vinhas Velhas Branco Whit...,Portuguese White,Other,1912 2011,1912,2011
38689,Portugal,An oak-aged wine that is rich and structured a...,Other,92,NaN,Douro,NaN,NaN,Roger Voss,@vossroger,1912 Winemakers 2009 Piorro Reserva Red (Douro),Portuguese Red,Other,1912 2009,1912,2009
41070,Portugal,"This classic blend of Douro grapes is soft, fr...",Other,87,NaN,Douro,NaN,NaN,Roger Voss,@vossroger,1912 Winemakers 2013 Piorro Branco Colheita Wh...,Portuguese White,Other,1912 2013,1912,2013
41483,Spain,"Tasty and smooth, this Rioja shows sweet black...",Other,86,9.0,Northern Spain,Rioja,NaN,NaN,NaN,Martinez Bujanda 1998 Conde de Valdemar Crianz...,Tempranillo,Other,1998 1998,1998,1998
46825,Spain,This slightly brooding red has a sweet and sou...,Other,86,14.0,Northern Spain,Rioja,NaN,NaN,NaN,Otanon 1998 Crianza 1998 (Rioja),Red Blend,Other,1998 1998,1998,1998
50821,South Africa,The palate is drenched in sweet oak and sugarp...,NaN,85,30.0,Other,NaN,NaN,NaN,NaN,Prospect 1870 1998 Cabernet Sauvignon (Robertson),Cabernet Sauvignon,Other,1870 1998,1998,1998
54720,Portugal,"Produced from old vines, this big and rounded ...",Other,93,NaN,Douro,NaN,NaN,Roger Voss,@vossroger,1912 Winemakers 2009 Encosta do Bocho Grand Re...,Portuguese Red,Other,1912 2009,1912,2009
61105,Portugal,"Made in tanks with no wood aging, this wine fr...",Other,87,NaN,Douro,NaN,NaN,Roger Voss,@vossroger,1912 Winemakers 2011 Piorro Colheita Red (Douro),Portuguese Red,Other,1912 2011,1912,2011


Après un rapide coup d'oeil, on en déduit que pour ces lignes spécifiques, on peut remplacer la valeur Year par la valeur Year_2 qui semble plus réaliste 

In [29]:
wine['year'] = np.where(((wine['year'] < '2000') & (wine['year_2'].notnull())), wine['year_2'], wine['year'])

In [30]:
wine.drop(['year_from_title', 'year_2'], axis=1, inplace=True)
wine.head(1)

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,year
id,,,,,,,,,,,,,,
0,Italy,"Aromas include tropical fruit, broom, brimston...",Other,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2013


## Title

Maintenant que nous avons récupéré l'année des vins, revenons à la colonne Title

Pour éviter la redondance des informations, on va supprimer les informations de localisation entre parenthèses ainsi que l'année de production  

In [31]:
wine['title'] = wine['title'].str.replace(r"\(.*?\)","")
wine.head(1)

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,year
id,,,,,,,,,,,,,,
0,Italy,"Aromas include tropical fruit, broom, brimston...",Other,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco,White Blend,Nicosia,2013


Pour l'année, on considère que le chiffre valide est forcément compris entre 1900 et 2020

In [32]:
wine['title'] = wine['title'].str.replace(r"(?<!\d)(?:19[0-9]\d|20[0-2]\d|2020)(?!\d)","")
wine.head(1)

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,year
id,,,,,,,,,,,,,,
0,Italy,"Aromas include tropical fruit, broom, brimston...",Other,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia Vulkà Bianco,White Blend,Nicosia,2013


In [33]:
wine.title.value_counts()

Siduri  Pinot Noir                                           36
L. Tramier & Fils  Collection                                31
La Crema  Pinot Noir                                         28
Roche de Bellene  Vieilles Vignes                            27
Williams Selyem  Pinot Noir                                  26
                                                             ..
1848 Winery  Second Generation Cabernet Sauvignon-Merlot      1
Casa Santos Lima  Gewurztraminer                              1
G D Vajra  Nostra Signora della Neve Sparkling                1
Hannah Nicole Vineyards  Limited Release Row 91 Malbec        1
Covey Run  Chardonnay )                                       1
Name: title, Length: 78247, dtype: int64

La suppression des infos a entraîné l'apparition de doubles espaces, corrigeons cela 

In [34]:
wine['title'] = wine['title'].str.replace('\s{2}', ' ')
wine.title.value_counts()

Siduri Pinot Noir                                  36
L. Tramier & Fils Collection                       31
La Crema Pinot Noir                                28
Roche de Bellene Vieilles Vignes                   27
Williams Selyem Pinot Noir                         26
                                                   ..
Parras Wines Terra Grande Red                       1
Miradou Miradou Rosé                                1
Sineann Block One Cabernet Sauvignon )              1
Domaine G. Metz Vieilles Vignes Gewurztraminer      1
Envy Price Vineyard Cabernet Sauvignon              1
Name: title, Length: 78211, dtype: int64

Maintenant, comme précédemment pour les autres variables, regroupons dans 'Other' les intitulés présents moins de 3 fois   

In [35]:
wine['title_count'] = wine.groupby('title')['title'].transform('count')
wine['other_title'] = np.where(wine['title_count']< 3, 'Other', ['title'])
wine.loc[wine['other_title'] == 'Other', 'title'] = 'Other'
wine.drop(['title_count', 'other_title'], axis=1, inplace=True)
wine['title'].value_counts()

Other                                                       81729
Siduri Pinot Noir                                              36
L. Tramier & Fils Collection                                   31
La Crema Pinot Noir                                            28
Roche de Bellene Vieilles Vignes                               27
                                                            ...  
Pierre Sparr Mambourg Grand Cru Gewurztraminer                  3
Billecart-Salmon NV Brut Réserve                                3
Chandon NV Étoile Brut Sparkling                                3
Dr. H. Thanisch  Bernkasteler Badstube Auslese Riesling         3
Steininger Loisiumweingarten Reserve Grüner Veltliner           3
Name: title, Length: 11910, dtype: int64

In [36]:
wine.head(1)

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,year
id,,,,,,,,,,,,,,
0,Italy,"Aromas include tropical fruit, broom, brimston...",Other,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Other,White Blend,Nicosia,2013


# Valeurs manquantes

Maintenant que nos catégories sont toutes valides pour chaque colonne, il est temps de gérer les valeurs manquantes

In [37]:
print(wine.isnull().any())

country                   True
description              False
designation               True
points                   False
price                     True
province                  True
region_1                  True
region_2                  True
taster_name               True
taster_twitter_handle     True
title                    False
variety                   True
winery                   False
year                      True
dtype: bool


Remplaçons d'abord les valeurs catégorielles manquantes par 'Other'

In [38]:
wine['country'].fillna('Other', inplace=True)
wine['designation'].fillna('Other', inplace=True)
wine['province'].fillna('Other', inplace=True)
wine['region_1'].fillna('Other', inplace=True)
wine['region_2'].fillna('Other', inplace=True)
wine['taster_name'].fillna('Other', inplace=True)
wine['taster_twitter_handle'].fillna('Other', inplace=True)
wine['variety'].fillna('Other', inplace=True)
print(wine.isnull().any())

country                  False
description              False
designation              False
points                   False
price                     True
province                 False
region_1                 False
region_2                 False
taster_name              False
taster_twitter_handle    False
title                    False
variety                  False
winery                   False
year                      True
dtype: bool


Maintenant les valeurs numériques, en commençant par le prix

In [39]:
print(wine['price'].mean())
print(wine['price'].median())
print(wine['price'].max())
print(wine['price'].min())

35.363389129985535
25.0
3300.0
4.0


Etant donné que nous avons des valeurs extrêmes pour le prix, prenons la médiane pour remplacer les valeurs manquantes 

In [40]:
wine['price'].fillna(wine['price'].median(), inplace=True)

Pour l'année, voyons laquelle est la plus présente pour l'indiquer comme base dans les valeurs manquantes

In [41]:
wine['year'].value_counts()

2013    15875
2012    15747
2014    15582
2011    12558
2010    12167
2015    10052
2009     9878
2008     7439
2007     7053
2006     5772
2016     3691
2005     3613
2004     1774
2000      759
2001      691
1999      641
2003      598
1998      551
2002      366
1997      302
1996       67
1995       50
1994       23
1992       16
2017       11
1989        6
1990        5
1988        5
1986        4
1985        4
1991        4
1963        3
1993        3
1952        2
1980        2
1966        2
1978        2
1927        2
1964        2
1984        2
1987        2
1983        2
1961        1
1973        1
1934        1
1941        1
1904        1
1974        1
1967        1
1935        1
1947        1
1969        1
1968        1
1945        1
1965        1
1976        1
1919        1
1982        1
1957        1
Name: year, dtype: int64

In [42]:
wine['year'].fillna('2013', inplace=True)
print(wine.isnull().any())

country                  False
description              False
designation              False
points                   False
price                    False
province                 False
region_1                 False
region_2                 False
taster_name              False
taster_twitter_handle    False
title                    False
variety                  False
winery                   False
year                     False
dtype: bool


In [43]:
wine.head(1)

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,year
id,,,,,,,,,,,,,,
0,Italy,"Aromas include tropical fruit, broom, brimston...",Other,87,25.0,Sicily & Sardinia,Etna,Other,Kerin O’Keefe,@kerinokeefe,Other,White Blend,Nicosia,2013
